In [1]:
from constants import *
import os
from dotenv import load_dotenv
import subprocess
import json
from pprint import pprint
from web3 import Web3
from web3.middleware import geth_poa_middleware
from eth_account import Account
from bit import wif_to_key, PrivateKeyTestnet
from bit.network import NetworkAPI

from pathlib import Path
from getpass import getpass
load_dotenv()

mnemonic = os.getenv('MNEMONIC')

In [2]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))

command = f'php hd-wallet-derive/hd-wallet-derive.php -g --mnemonic="{mnemonic}" --cols=path,address,privkey,pubkey --numderive=3 --format=jsonpretty'

In [3]:
p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
(output, err) = p.communicate()
p_status = p.wait()
keys = json.loads(output)


In [4]:
def derive_wallets(coins=[],numderive=3):
    coin_dict ={}
    for coin in coins:
        command = f'php hd-wallet-derive/hd-wallet-derive.php -g --mnemonic="{mnemonic}" --cols=all --coin={coin} --numderive={numderive} --format=jsonpretty'
        p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
        (output, err) = p.communicate()
        p_status = p.wait()
        keys = json.loads(output)
#         coin_dict[coin]=coin
        coin_dict[coin]=keys
  
    return coin_dict

In [18]:
coins=derive_wallets(coins=[BTCTEST, ETH])

In [19]:
# We sent 0.03103952 bitcoins to address
# mupLK82DqRXdxcziEPqNaHwVC5jjxZxGkL

# tx: 1706485f19458d82393dda5f04378bf40b4cdf9252a8e475913362430e13835e
# Send coins back, when you don't need them anymore to the address

# mv4rnyY3Su5gjcDNzbMLKBQkBicCtHUtFB

# We sent 0.02530328 bitcoins to address
# mupLK82DqRXdxcziEPqNaHwVC5jjxZxGkL

# tx: a6d8a54f13041aa404271bea065c573a615f66661def8eaada130df7dbeb44df
# Send coins back, when you don't need them anymore to the address

# mv4rnyY3Su5gjcDNzbMLKBQkBicCtHUtFB



In [20]:
coins['eth'][0]


{'path': "m/44'/60'/0'/0/0",
 'address': '0xf3d5d4201ff0bd522CF613B23E2122f5AC393088',
 'xprv': 'xprvA3seeSEcyAiEfKaFWpGDHgBdrfreBcsER1EiCHChfc5YRSvpJAqg6feLBcmBXYHkh3cQ86cycPBASLqLEyeBye4DGrd8eVVuNFRxJZ2NdGB',
 'xpub': 'xpub6Gs13wmWoYGXsoeicqoDep8NQhh8b5b5nEAJzfcKDwcXJFFxqi9veTxp2srPGbWhxLoHgE7q5y95nP5Ehq8Av62EPZMWffS8hrcLbjqdWTN',
 'privkey': '0xc48be520b745fb04809109b84b1b6b73984462434ef525aa43c05f23cf0f3a53',
 'pubkey': '02cc594b3fbf3b714f14b807f69097f3902fef3990a9216f2fc0148e8ca85e7e93',
 'pubkeyhash': 'f7237674afc1aa75e253c377586526470ba56a01',
 'index': 0}

In [5]:
def priv_key_to_account(coin):
    coins=derive_wallets(coins=[BTCTEST, ETH])
    if coin == 'eth':
        with open(Path("UTC--2019-11-20T02-42-15.118067200Z--b8e45f5aef7bcaa100d5cbd4a69caa7816d639c8")) as keyfile:
            encrypted_key = keyfile.read()
            private_key = w3.eth.account.decrypt(
                encrypted_key, getpass("Enter keystore password: ")
            )
            key = Account.from_key(private_key)
    elif coin == 'btc-test':
        key = PrivateKeyTestnet(coins['btc-test'][0]['privkey'])
    return key
    

In [32]:
def create_tx(coin, account, to, amount):
    if coin == 'eth':
        gasEstimate = w3.eth.estimateGas(
        {"from": account.address, "to": to, "value": amount}
    )
        info = {
        "from": account.address,
        "to": to,
        "value": amount,
        "gasPrice": w3.eth.gasPrice,
        "gas": gasEstimate,
        "nonce": w3.eth.getTransactionCount(account.address),
    }
    elif coin == 'btc-test':
        info = PrivateKeyTestnet.prepare_transaction(str(account.address), [(to, amount, BTC)])
 
    return info        

In [33]:
def send_tx(coin, account, to, amount):
    tx = create_tx(coin, account, to, amount)
    if coin == 'eth':
        signed_tx = account.signTransaction(tx)
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction).hex()
    elif coin == 'btc-test':
        signed_tx = account.sign_transaction(tx)
        result = NetworkAPI.broadcast_tx_testnet(signed_tx)
    return result
    

In [38]:
def check_balance(coin):
    if coin == ETH:
        balance = w3.eth.getBalance(priv_key_to_account(ETH).address)
    elif coin == BTCTEST:
        balance = priv_key_to_account(BTCTEST).get_balance()
    return balance

In [40]:
# key1.get_balance("btc")
# key1.get_unspents()
# key.create_transaction(outputs,unspents=key.get_unspents())
# key.send(outputs,unspents=key.get_unspents())
# w3.eth.getBalance(key.address)
# PrivateKeyTestnet.prepare_transaction('mupLK82DqRXdxcziEPqNaHwVC5jjxZxGkL', [('mweZvwfg4J3WVYCjXeELCVWouL2LAuuE9b', .01, BTC)])
check_balance(BTCTEST)

'2878496'

In [10]:
# #BTC KEYS:
# key = priv_key_to_account(coins['btc-test'][0]['privkey'], 'btc-test')

# key1 = priv_key_to_account(coins['btc-test'][1]['privkey'],'btc-test')
# key1.address

'mupLK82DqRXdxcziEPqNaHwVC5jjxZxGkL'

In [15]:
send_tx('btc-test',priv_key_to_account(BTCTEST), 'mupLK82DqRXdxcziEPqNaHwVC5jjxZxGkL', .001)

In [16]:
priv_key_to_account(BTCTEST).get_transactions()

['3f16f7fa243fc2b88f72fcdd97426199a561939b68599f332c6f84e8a8fcd4b4',
 'd76a82fcc7345bd60a5576e1e3995682d8e9c635cd96ed7470a09ccd1e501436',
 '1cd47b20f7fa18f66fd6e96c491a258015abfc5f3e169b9153d5d0276f2db38c',
 'cee70f8a36458d3692422ce52ba83575d334bac3e6da9f0de9d6b006d14180a1',
 '60e0b9cbd7402961b5bfeb416fb7fa9d58ac31723467d051798aa35f83877332',
 'ca4f9ab7253959659a50af377c2514fd083a4dab2aa0919f33ab8dce133d88d0']

In [34]:
send_tx(ETH,priv_key_to_account(ETH), '0xf3d5d4201ff0bd522CF613B23E2122f5AC393088', 1)

Enter keystore password: ········


'0x6c1ded73012b60b670268a99d915987958bbbe2edcca98ccd8acc6d4863275ac'

In [35]:
print(w3.eth.getTransaction("0x6c1ded73012b60b670268a99d915987958bbbe2edcca98ccd8acc6d4863275ac"))

AttributeDict({'blockHash': HexBytes('0x3d578dd498f5884a7bcdfc12015d5ce8ffaa806236a8218b8b1f0d9233377653'), 'blockNumber': 13929, 'from': '0xB8E45F5aeF7BCaa100D5cBd4a69caa7816d639C8', 'gas': 21000, 'gasPrice': 1000000000, 'hash': HexBytes('0x6c1ded73012b60b670268a99d915987958bbbe2edcca98ccd8acc6d4863275ac'), 'input': '0x', 'nonce': 1, 'to': '0xf3d5d4201ff0bd522CF613B23E2122f5AC393088', 'transactionIndex': 0, 'value': 1, 'v': 27, 'r': HexBytes('0xc9110edb367cd890ea4ed5c281a438a7f8ae233a206b26a417db7842bf5db339'), 's': HexBytes('0x243856bced429d0e7ce42f60d2537c41db2f67ab0f63c92672e79c66c7526ccc')})


In [37]:
w3.eth.getBalance(priv_key_to_account(ETH).address)

Enter keystore password: ········


904625697166532776746648320380374280103671755200316906559262837061821325309